In [1]:
%pylab
import matplotlib.dates as mdates
from moviepy.video.io.bindings import mplfig_to_npimage
import moviepy.editor as mpy
import pandas

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [15]:
# Load generated movie
rotmov = mpy.VideoFileClip("movtest.avi")

# determine sizes and offsets for plotting the rotating data
fsize = rotmov.get_frame(0).shape
rotshape = rotmov.get_frame(0)[:,700:1171].shape
rotw = rotshape[1]
roth = rotshape[0]
rotoffh = int((1080-roth)/2)-1
rotoffw = int(1920/2)+int(1920/6)-1

# Background color
bgcolor = array([126, 126, 126])
scolor = bgcolor/255 # Scaled to 255

df = pandas.read_csv("/home/maxc/Desktop/wormhole-scripts/PZ103_PressureLog.csv", sep=";", skiprows=4)
dP = df.iloc[:-1,8:10]
dP = dP.replace(regex=r'\ [A|P]M*', value='')
dP['Time SN:702643 (S30X):CH0'] = pandas.to_datetime(dP['Time SN:702643 (S30X):CH0'])
dP = dP.set_index('Time SN:702643 (S30X):CH0')
dP['SN:702643 (S30X):CH0 bar'] = pandas.to_numeric(dP['SN:702643 (S30X):CH0 bar'])

ss = pandas.read_csv("/home/maxc/Desktop/wormhole-scripts/PZ103_ScanTimes.txt", skiprows=3)
ss['StartTime'] = pandas.to_datetime(ss['StartTime'])
ss['EndTime'] = pandas.to_datetime(ss['EndTime'])




# Pressure figure
pfig= figure(figsize=(9.6,5.4))
pax = pfig.gca()

pax.set_xlim([dP.index[6],dP.index[-3]])
pax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
dP[dP.index[6]:dP.index[-2]].plot(ax=pax, legend=False)
pax.set_xlabel("Time", fontsize=20)
pax.set_ylabel("dP (atm)", fontsize=20)

vert0 = pax.axvline(ss.values[0,0], color='k')
vert1 = pax.axvline(ss.values[0,1], color='k')
span = pax.axvspan(ss.values[0,0], ss.values[0,1], alpha=0.5, color=scolor)
tight_layout()

pimg = mplfig_to_npimage(pfig) # plot to image
#close(pfig)

# Create an initial frame buffer
fbuffer = ones((1080,1920,3)).astype(uint8)*126
rotframe = rotmov.get_frame(0)[:,700:1171]
fbuffer[rotoffh:rotoffh+roth,rotoffw:rotoffw+rotw,:] = rotframe
fbuffer[:540,:960,:] = pimg

#imshow(fbuffer)
#show()

def mkframe(t):
    global span
    if t%2==0.0 and t!=0:
        vert0.set_xdata(ss.values[int(t/2),0])
        vert1.set_xdata(ss.values[int(t/2),1])
        span.remove()
        span = pax.axvspan(ss.values[int(t/2),0], ss.values[int(t/2),1], alpha=0.5, color=scolor)
        draw()
        pimg = mplfig_to_npimage(pfig)
        fbuffer[:540,:960,:] = pimg
        print("Time slice " + str(t/2+1))
        #print(t)

    rotframe = rotmov.get_frame(t)[:,700:1171]
    fbuffer[rotoffh:rotoffh+roth,rotoffw:rotoffw+rotw,:] = rotframe
    return fbuffer

In [16]:
animation = mpy.VideoClip(mkframe, duration=98)
animation.write_videofile("CropTest.avi", fps=60, codec="libx264", logger=None)

Time slice 2.0
Time slice 3.0
Time slice 4.0
Time slice 5.0
Time slice 6.0
Time slice 7.0
Time slice 8.0
Time slice 9.0
Time slice 10.0
Time slice 11.0
Time slice 12.0
Time slice 13.0
Time slice 14.0
Time slice 15.0
Time slice 16.0
Time slice 17.0
Time slice 18.0
Time slice 19.0
Time slice 20.0
Time slice 21.0
Time slice 22.0
Time slice 23.0
Time slice 24.0
Time slice 25.0
Time slice 26.0
Time slice 27.0
Time slice 28.0
Time slice 29.0
Time slice 30.0
Time slice 31.0
Time slice 32.0
Time slice 33.0
Time slice 34.0
Time slice 35.0
Time slice 36.0
Time slice 37.0
Time slice 38.0
Time slice 39.0
Time slice 40.0
Time slice 41.0
Time slice 42.0
Time slice 43.0
Time slice 44.0
Time slice 45.0
Time slice 46.0
Time slice 47.0
Time slice 48.0
Time slice 49.0


In [10]:
from matplotlib.backends.backend_pdf import PdfPages


df = pandas.read_csv("/home/maxc/Desktop/wormhole-scripts/PZ103_PressureLog.csv", sep=";", skiprows=4)
dP = df.iloc[:-1,8:10]
dP = dP.replace(regex=r'\ [A|P]M*', value='')
dP['Time SN:702643 (S30X):CH0'] = pandas.to_datetime(dP['Time SN:702643 (S30X):CH0'])
dP = dP.set_index('Time SN:702643 (S30X):CH0')
dP['SN:702643 (S30X):CH0 bar'] = pandas.to_numeric(dP['SN:702643 (S30X):CH0 bar'])

ss = pandas.read_csv("/home/maxc/Desktop/wormhole-scripts/PZ103_ScanTimes.txt", skiprows=3)
ss['StartTime'] = pandas.to_datetime(ss['StartTime'])
ss['EndTime'] = pandas.to_datetime(ss['EndTime'])

bgcolor = array([126, 126, 126])
scolor = bgcolor/255 # Scaled to 255

with PdfPages('PressureCurveFrames.pdf') as pdf:
    pfig= figure(figsize=(9.6,5.4))
    pax = pfig.gca()

    pax.set_xlim([dP.index[6],dP.index[-3]])
    pax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    dP[dP.index[6]:dP.index[-2]].plot(ax=pax, legend=False)
    pax.set_xlabel("Time", fontsize=20)
    pax.set_ylabel("dP (atm)", fontsize=20)

    vert0 = pax.axvline(ss.values[0,0], color='k')
    vert1 = pax.axvline(ss.values[0,1], color='k')
    span = pax.axvspan(ss.values[0,0], ss.values[0,1], alpha=0.5, color=scolor)
    frtext = pfig.text(0.93,0.9, "02", fontsize=20)
    tight_layout()
    pdf.savefig()

    for t in arange(48)+1:
        vert0.set_xdata(ss.values[int(t),0])
        vert1.set_xdata(ss.values[int(t),1])
        span.remove()
        span = pax.axvspan(ss.values[int(t),0], ss.values[int(t),1], alpha=0.5, color=scolor)
        frtext.set_text(str(t+2).zfill(2))
        draw()
        pdf.savefig()
        